# Pandas 1
![Pandas](http://i2.cdn.turner.com/cnnnext/dam/assets/111017060721-giant-panda-bamboo-story-top.jpg "Pandas")

Let's say I have data as a table (again, using the car parts example):

Part name | Number of units | Price per unit | Total unit price
---- |:----:| ----:| ----:
Wheels | 4 | 500 | 2000
Doors | 4 | 200 | 800
Windows | 4 | 100 | 400
Engine | 1 | 2000 | 2000
Body | 1 | 5000 | 5000


There is no way to represent this entire table nicely using the data structures we've seen so far.

Pandas allows us to play with such tables easily.

The first step in using Pandas is to import the module. We will use two statements at the beginning of every pandas code:

In [2]:
from pandas import Series, DataFrame
import pandas as pd

This does two things:

1. With the "import pandas as pd", we can call pandas functions using pd.function-name (e.g., pd.read_csv)
2. Two particular data structures will be used very often: Series and DataFrame. Instead of having to say pd.Series, we import these specifically, so we can now just call Series. Ditto for DataFrame.


This lecture will focus on these two structures:

* Series, and
* DataFrame

## Series

A Series is a list-like object, but with a few differences.

In [3]:
unit_prices = [500, 200, 100, 2000, 5000]
obj = Series(unit_prices) # initializing a Series from a list
obj

0     500
1     200
2     100
3    2000
4    5000
dtype: int64

A Series has three things:
1. An _index_ (here, 0, 1, 2, 3). We can access individual elements of the Series with this index. Lists also have this implicit index, but with a Series, this index can be arbitrary.
2. *Values* corresponding to these index items (here, 500, 200, 100, ...).
3. *Dtype*, which is the type of the values (in this case, int64). Recall that a list can combine items of arbitrary types. A Series has items of the same type.

In [ ]:
obj.values # gives the values in a Series

In [ ]:
obj.index

In [ ]:
obj.dtype

Let's create a more interesting Series.

In [ ]:
part_names = ['Wheels', 'Doors', 'Windows', 'Engine', 'Body']
unit_price_series = Series(unit_prices, index=part_names)
unit_price_series

Instead of creating a Series from two lists (the values, and the indices), we can also create it from a dictionary.

In [ ]:
obj2 = Series({'Wheels':500, 'Doors':200, 'Windows':100, 'Engine':2000, 'Body':5000})
obj2

### Accessing elements

Series combine properties of lists and dictionaries:

* The Series values are in a list-like form, and can be accessed just like a list.
* The Series index provides _keys_ to access the corresponding values, just like a dictionary.

Thus, a Series allows us to use both list-like and dictionary-like access.

#### Dictionary-style access to Series

In [ ]:
# Dictionary-style access
unit_price_series['Windows']

In [ ]:
unit_price_series[['Body', 'Doors', 'Windows']]

In [ ]:
# we can search within the index, just like for dictionary keys
'Body' in unit_price_series

#### List-style access

In [ ]:
unit_price_series

In [ ]:
# List-style access
unit_price_series[1]

In [ ]:
unit_price_series[3:]

Finally, we can combine dictionary-style access with list-like slicing.

In [ ]:
unit_price_series['Wheels':'Windows']

### Difference from dictionary

There are two main differences from a dictionary.


* In a Python dictionary, there is no ordering on the keys.
    * You cannot say, dict[key1:key5]. 
    * However, list-like slicing on the index is allowed for Series. 
    * That is why _the order of the indices matter_.

* In a dictionary, all the keys have to be distinct; you can only have one value per key.
    * However, that is not so for Series.

In [ ]:
labels_copy = ['Wheels'] * 5  # recall: the '*'-operator repeats list items
labels_copy

In [ ]:
obj3 = Series(unit_prices, index=labels_copy)
obj3

The index now has repeated items, so there are multiple values for the same index. 

In [ ]:
obj3['Wheels']  # Returns a Series; not just one value like for dictionaries!

### Filtering a Series

One of the important functions that can be performed on a Series is filtering. Suppose we want all units priced less than some amount, say, 400. How do we do it?

In [ ]:
mask = (unit_price_series < 400)
mask

This gives a Boolean Series where we have the same index, but the values are True (if value < 400) or False (value >= 400). This is often called a boolean **mask**.

The mask can be used to select out items from a Series.

In [ ]:
unit_price_series[mask]

In [ ]:
unit_price_series  # We already have the series of unit prices

In [ ]:
# Let's create another Series of number-of-units for each car part
num_units_series = Series({'Doors':4, 'Windows':4, 'Wheels':4, 'Engine':1, 'Body':1})
num_units_series

*Example*: Find the unit prices of all car parts of which we only need 1 unit.

In [ ]:
mask = (num_units_series == 1)  # Recall: == is equality condition
mask

In [ ]:
unit_price_series[mask]

**NOTE:** The _order_ of parts in unit_price_series and num_units_series are different! However, this is where the index is useful; pandas doesn't use the ordering, it uses the index to figure out how to apply the mask to unit_price_series.

### Operations on Series

Obvious things work.

In [ ]:
# Increase unit prices by 3% for inflation
unit_price_series * 1.03

If you want to apply some function to the Series, use the map() method of Series.

In [ ]:
# Find square-roots of all unit prices
import math
unit_price_series.map(math.sqrt)

We can also get aggregate statistics of a Series.

In [ ]:
print('Mean =', unit_price_series.mean())  # Average unit price
print('Variance =', unit_price_series.var()) # Variance of unit prices
print('Max =', unit_price_series.max(), ' for car part =', unit_price_series.idxmax())

*Example*: Find all car parts whose unit price is at least 10% of the priciest part.

In [ ]:
unit_price_series[unit_price_series >= 0.1 * unit_price_series.max()]

In [ ]:
unit_price_series[unit_price_series >= 0.1 * unit_price_series.max()]

We can also combine two Series in obvious ways.

In [ ]:
unit_prices_second_car = Series({'Wheels':600, 'Doors': 400, 'Windows':100, 'Engine':5000, 'Body':10000})
print('Second car:')
print(unit_prices_second_car)
print
print('First car:')
print(unit_price_series)

*Example*: Find the average unit price for each car part.

In [ ]:
# Average unit price for each car part
(unit_price_series + unit_prices_second_car) / 2

### Missing values

Real-world data is often full of missing or incorrect values.
One of the advantages of pandas is that it makes dealing with missing values relatively painless. 

In [ ]:
# Let's ask for a missing car part
# unit_price_series[['Engine', 'Transmission', 'Body']] throws an error
# Instead, we use the "reindex" method.

missing_series = unit_price_series.reindex(['Engine', 'Transmission', 'Body'])
missing_series

The 'NaN' stands for "Not A Number", and this is how pandas denotes missing values. 

Another common situation is when we process two series with mismatched indices.

There are three types of operations we can do with missing values:

* find the items with missing values,
* drop them from our Series, or
* fill the missing values with a value of our choice.

In [ ]:
# Find missing elements
mask = missing_series.isnull()
mask

In [ ]:
missing_series[mask]

In [ ]:
# Drop missing elements
missing_series.dropna()

In [ ]:
# Fill missing values
missing_series.fillna(-1)

*Example*: Replace missing values with the mean.

How do we do it?

In [ ]:
missing_series.fillna(missing_series.mean())

In [ ]:
missing_series.fillna(missing_series.mean())

### Summary

A Series allows us to attach an index to a list. This has several benefits:

* The index allows dictionary-like access to the list items, in addition to the usual list-like access.
* Pandas lets us combine two Series by "matching up" their indices.
* Finally, there are lots of helper functions to modify values, deal with missing values, compute statistics and such.

However, it still leaves much to be desired.

* We still cannot represent the entire car parts table using just a series
    * We need multiple series

A DataFrame is just that.

## DataFrame

Roughly, DataFrame = combination of Series sharing the same index. 

For instance, our Car Parts table can be thought of as three series (unit price, number of units, and total unit price) on the same index (car part name).

In [ ]:
data = {'unit price': [500, 200, 100, 2000, 5000], 'number of units':[4, 4, 4, 1, 1]}
print('data =', data)
print('part_names =', part_names)

car_table = DataFrame(data, index=part_names)
car_table

Thus, each column of the DataFrame is a Series, and all the series share the same index.

### Accessing elements

We can easily get the individual series that form this DataFrame.

In [ ]:
car_table['number of units']

We can also add new columns.

In [ ]:
car_table['Total unit price'] = car_table['number of units'] * car_table['unit price']
car_table

Accessing _rows_ is a little different.

In [ ]:
# Get all information about Windows
car_table.loc['Windows']

Notice that this also gives us a Series; it is just that row written out as a Series.

What happens if you want two rows?

In [ ]:
# Get two rows
car_table.loc[['Engine', 'Body']]

In [ ]:
# Get total unit price of just Wheels and Doors
car_table.loc[['Wheels', 'Doors'], ['Total unit price']]

We can also use list-like indexing for the rows

In [ ]:
# First two rows and columns
car_table.iloc[:2, :2]

We can again use masks.

In [ ]:
# Let's add a second car.
car_table['car-2 unit price'] = [300, 400, 500, 3000, 4000]
car_table['car-2 Total unit price'] = car_table['car-2 unit price'] * car_table['number of units']
car_table

*Example*: Find units for which car-2 is pricier than the first car.

In [ ]:
mask = (car_table['car-2 unit price'] > car_table['unit price'])
car_table[mask]

In [ ]:
# To flip the index and columns
car_table.T   # T is short-form for "transpose", which flips rows and columns of a matrix

### Reading from CSV files

Most often, you will have data in a tabular form somewhere and you'll read from it. Pandas allows us to easily build DataFrames from CSV files.

In [ ]:
!cat Intro_4_data/CarParts.csv

In [ ]:
df = pd.read_csv('Intro_4_Data/CarParts.csv')
df

This creates a data frame as desired, but the index is the _default_ index.

In [ ]:
print(df.index)

We want to set the 'Part name' to be the index. We do this via set_index(). 

In [ ]:
df.set_index('Part name', inplace=True)
df

In [ ]:
print(df.index)

In [ ]:
print(df.index.values)

### Operations on a DataFrame

It is easy to select a Series, and apply a formula to that Series.

In [ ]:
# mean unit price of car parts
df['Price per unit'].mean()

We can also apply the same function to all columns.

*Example*: Find the range of values (max - min) for each of the columns.

In [ ]:
def get_column_range(x):
    # x here is a Series
    return x.max() - x.min()

# "Apply" this range function to each column of the DataFrame
df.apply(get_column_range)

Another common operation is sorting the entire DataFrame. There are two methods for this:

* *sort_index()*, and
* *sort_values()*

In [ ]:
# Sort the DataFrame by its index.
df.sort_index()

In [ ]:
# Sort the DataFrame by price per unit
df.sort_values(by='Price per unit')

### Summary

A DataFrame helps organize several Series together. Each Series becomes a column of a table, and they are all linked via the same index.

* Read in a table using pd.read_csv  (or pd.read_table(); do help(pd.read_table)!)
* Access a column by df['Number of units']
* Access a row by df.loc['Windows'] or df.iloc[0]
* Change the index using df.set_index('Price per unit', inplace=True)
* Apply arbitrary functions using apply()
* In general, use Series methods after selecting out a column of the DataFrame.